In [1]:
import numpy as np

from dftpy.constants import ENERGY_CONV
from edftpy import io
from edftpy.functional import LocalPP, KEDF, Hartree, XC
from edftpy.optimizer import Optimization
from edftpy.evaluator import EmbedEvaluator, TotalEvaluator
from edftpy.subsystem.subcell import SubCell, GlobalCell
from edftpy.interface import init_graphtopo
from edftpy.mpi import MP, sprint
from edftpy.engine.driver import DriverKS
from edftpy.engine.engine_qe import EngineQE
from edftpy.utils.common import Field, Functional, AbsFunctional

In [2]:
def get_optimizer(cellfile, subkeys, indices):
    #-----------------------------------------------------------------------
    pplist = {'H' : './H_ONCV_PBE-1.2.upf', 'O' : './O_ONCV_PBE-1.2.upf'}
    ecut = 1200*ENERGY_CONV["eV"]["Hartree"]
    cellsplit = [0.5, 0.5, 0.5]
    #-----------------------------------------------------------------------
    ions = io.ase_read(cellfile)
    graphtopo = get_graphtopo([1,]*len(subkeys), parallel = True)
    gsystem = get_gsystem(ions, graphtopo, pplist, ecut)
    drivers = []
    for i, keysys in enumerate(subkeys):
        if graphtopo.isub != i and graphtopo.is_mpi:
            driver = None
        else :
            index = indices[i]
            driver = get_driver(keysys, ions, gsystem.grid, pplist, index, cellsplit, graphtopo)
        drivers.append(driver)

    graphtopo.build_region(grid=gsystem.grid, drivers=drivers)
    opt = Optimization(drivers = drivers, gsystem = gsystem, options={'econv': 1E-6*ions.nat})
    return opt

In [3]:
def get_gsystem(ions, graphtopo, pplist, ecut):
    mp_global = MP(comm = graphtopo.comm, parallel = graphtopo.is_mpi, decomposition = graphtopo.decomposition)
    gsystem = GlobalCell(ions, ecut = ecut, mp = mp_global, graphtopo = graphtopo)
    total_evaluator = get_total_evaluator(ions, gsystem.grid, pplist)
    gsystem.total_evaluator = total_evaluator
    return gsystem

In [4]:
def get_graphtopo(nprocs, parallel = False):
    graphtopo = init_graphtopo(parallel)
    graphtopo.distribute_procs(nprocs)
    return graphtopo

In [5]:
def ext_functional(density,**kwargs):
    factor = (3.0 / 10.0) * (5.0 / 3.0) * (3.0 * np.pi ** 2) ** (2.0 / 3.0)
    potential = factor * np.cbrt(density* density)
    energy=(potential*density).sum()*density.grid.dV*3.0/5.0
    obj = Functional(name = 'EXT0', energy=energy, potential=potential)
    return obj

In [6]:
class ExtFunctional(object):
    def __init__(self, vext=None, **kwargs):
        self.vext=vext
        
    def __call__(self, density, **kwargs):
        potential=self.vext
        energy=(potential*density).sum()*density.grid.dV
        obj = Functional(name = 'EXT1', energy=energy, potential=potential)
        return obj

In [7]:
def get_total_evaluator(ions, grid, pplist):
    xc_kwargs = {'xc' : 'PBE'}
    ke_kwargs = {'kedf' : 'GGA', 'k_str' : 'revAPBEK'}
    pseudo = LocalPP(grid = grid, ions=ions, PP_list=pplist)
    hartree = Hartree()
    xc = XC(**xc_kwargs)
    ke = KEDF(**ke_kwargs)
    funcdicts = {'XC' :xc, 'HARTREE' :hartree, 'PSEUDO' :pseudo, 'KE' :ke, 'EXT0': ext_functional}
    total_evaluator = TotalEvaluator(**funcdicts)
    return total_evaluator

In [8]:
def get_embed_evaluator(subcell):
    xc_kwargs = {'xc' : 'PBE'}
    ke_kwargs = {'kedf' : 'GGA', 'k_str' : 'revAPBEK'}
    xc = XC(**xc_kwargs)
    ke = KEDF(**ke_kwargs)
    # External Potential--------------------------------------------
    vext = Field(grid=subcell.grid)
    vext[:]= -1E-6
    extobj = ExtFunctional(vext)
    #---------------------------------------------------------------
    emb_funcdicts = {'XC' :xc, 'KE' :ke, 'EXT0': ext_functional, 'EXT1': extobj}
    embed_evaluator = EmbedEvaluator(**emb_funcdicts)
    return embed_evaluator

In [9]:
def get_driver(prefix, ions, grid, pplist, index, cellsplit, graphtopo):
    mp = MP(comm = graphtopo.comm_sub, decomposition = graphtopo.decomposition)
    subcell = SubCell(ions, grid, index = index, cellsplit = cellsplit, mp = mp)
    # given a negative value which means will get from driver
    subcell.density[:] = -1.0
    embed_evaluator = get_embed_evaluator(subcell)
    cell_params = {'pseudopotentials' : pplist}
    params = {'system' : {'ecutwfc' : 600*ENERGY_CONV["eV"]["Hartree"]*2}}
    margs= {
            'evaluator' : embed_evaluator,
            'prefix' : prefix,
            'subcell' : subcell,
            'cell_params': cell_params,
            'params': params,
            'exttype' : 3, # 3 is XC embedded, 7 is without XC
            'mixer' : 0.7
            }
    engine = EngineQE()
    driver = DriverKS(engine = engine, **margs)
    return driver

In [10]:
# cellfile = 'h2o_2.xyz'
# subkeys = ['sub_ks_0', 'sub_ks_1']
# indices = [[0, 1, 2], [3, 4, 5]]
cellfile = 'h2o_1.xyz'
subkeys = ['sub_ks_0']
indices = [[0, 1, 2]]
opt = get_optimizer(cellfile, subkeys, indices)

********************************************************************************
Parallel version (MPI) on        1 processors
eDFTpy Version : 0.0.1
 DFTpy Version : 0.0.2
********************************************************************************
GlobalCell grid [72 72 72]
setting key: H -> ./H_ONCV_PBE-1.2.upf
setting key: O -> ./O_ONCV_PBE-1.2.upf


In [11]:
def print_density(opt=opt):
    n=opt.gsystem.density.integral()
    if opt.gsystem.graphtopo.comm.rank==0:
        print('**Charge of global system: {}'.format(n))

In [12]:
def print_density_driver_all(opt=opt):
    for isub, driver in enumerate(opt.drivers):
        if driver is None: continue
        if driver.comm.rank==0:
            n=driver.density.integral()
            print('****Charge of driver {} is : {}'.format(isub,n))

In [13]:
def print_ext_energy(opt=opt, isub=0):
    driver=opt.drivers[isub]
    if driver is None: return
    if driver.comm.rank==0:
        energy=ext_functional(driver.density).energy
        print('******Ext energy of {} driver : {}'.format(isub,energy))

In [14]:
opt.attach(print_density, interval=1)
opt.attach(print_density_driver_all, interval=2)
opt.attach(print_ext_energy, interval=1)
opt.optimize()

Begin optimize
Optimization options :
{'econv': 3e-06,
 'maxiter': 80,
 'maxtime': 0,
 'ncheck': 2,
 'olevel': 2,
 'pconv': 3.0000000000000004e-08,
 'pconv_sub': array([3.e-08]),
 'sdft': 'sdft'}
Update density : 8.000000000000004
          Step    Energy(a.u.)            dE              dP        dC        Time(s)         
**Charge of global system: 8.000000000000004
****Charge of driver 0 is : 7.9999999999999885
******Ext energy of 0 driver : 9.358214438136557
**Charge of global system: 8.000000000000004
******Ext energy of 0 driver : 10.26557518549203
Norm of reidual density : 
[0.00346538]
Energy of reidual density : 
[0.32330905]
----------------------------------------------------------------------------------------------------
   Embed: 1       8.177912587980E+00      8.177913E+00    3.23E-01  3.47E-03  5.544879E-01    
----------------------------------------------------------------------------------------------------
**Charge of global system: 8.000000000000005
****Charge of d